### Pomožne funkcije pri izdelavi pajka
 

In [45]:
import requests
from bs4 import BeautifulSoup



def robots_context(text):
    result_dict = {}
    lines = text.split('\n')
    sitemaps = []
    for line in lines:
        if line.startswith('Sitemap:'):
            if len(line.split())>1: sitemaps.append(line.split()[1])
        elif line.startswith('User-agent:') and line.split()[1] == '*':
            disallows = []
            allows = []
            for agent in lines[lines.index(line)+1:]:
                print(agent)
                if agent.strip() and  not agent.strip().startswith(('Disallow:', 'Allow:', '#')):
                    break
                if agent.startswith('Disallow:'):
                    if len(agent.split())>1: disallows.append(agent.split()[1])        
                if agent.startswith('Allow:'):
                    if len(agent.split())>1: allows.append(agent.split()[1]) 
            result_dict['User-agent'] = {'Disallow': disallows, 'Allow': allows}
        elif line.startswith('Crawl-delay:'):
            if len(line.split())>1: result_dict['Crawl-delay'] = line.split()[1]
        else:
            continue
    result_dict['Sitemap'] = sitemaps
    return result_dict



def url_sitemap(url):
    #za vstavlen url osnovne domene dobi vse povezave na strani
    response = requests.get(url + '/robots.txt')
    sitemap_list = robots_context(response.text)
    print(sitemap_list['Sitemap'])
    urls = []
    for sitemap in sitemap_list['Sitemap']:
        urls.extend(parse_sitemap(sitemap))
    return urls


def parse_sitemap(url_sitemap):
    #dobi urlje od sitmap xml datoteke
    urls = []

    #če je ni
    if url_sitemap is  None:
        return []

    # Fetch the sitemap XML file
    response = requests.get(url_sitemap)
    if response.status_code == 200:
        #preoblikujemo v leš
        soup = BeautifulSoup(response.content, 'xml')
        

    
        #išče <sitemap>, ki nakazuje na .xml datoteke
        sitemap = soup.find('sitemap')
        if sitemap is not None:
            
            
            for sitemap in soup.sitemapindex.find_all('sitemap'):
                # Extract the URLs from the sitemap
                loc = sitemap.loc.text
                
                #print(f'Location: {loc}')

                #če so xml datoteke
                if loc.endswith(".xml") or loc.endswith(".XML"):
                    urls.extend(parse_sitemap(loc))
                    
            #kliče sebe s parametrom, ki vsebuje ime xml datoteke z url-ji
        
        else:
            #datotka vsebuje url-je
            urlset = soup.find('urlset')
            if urlset is not None:
                url = soup.find('url')
                if url is not None:
                    urls.extend([element.text for element in soup.find_all('loc')])
       
  
    return urls


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url = 'https://okusno.je/'  # Replace example.com with the actual domain


response = requests.get(url, headers=headers)
if response.status_code == 200:  # Check if the response is successful
    #print(len(url_sitemap(url)))
    resp = requests.get(url + '/robots.txt', headers=headers)

    print(robots_context(resp.text))
else:
    print('Failed to retrieve robots.txt')





Disallow: /iskanje
Disallow: /bin/
Disallow: /lbin/
Disallow: /adserver/
Disallow: *modal:*
Crawl-delay: 2
{'User-agent': {'Disallow': ['/iskanje', '/bin/', '/lbin/', '/adserver/', '*modal:*'], 'Allow': []}, 'Crawl-delay': '2', 'Sitemap': ['https://www.24ur.com/sitemaps/sites/10010']}


In [44]:
from urllib.parse import urlparse

websites = set()
web = "https://docs.python.org/3/library/"

# Odstrani duplikate glede na url:
#    HTTP vs HTTPS
#    www vs non-www
#    Trailing slash: example.com vs example.com/
def is_unique(website):
    parsed = urlparse(website)
    #odstrani zadnji / če obstaja
    path = parsed.path[:-1] if (parsed.path[-1] == "/") else parsed.path
    #docs.python.org/3/library
    url = parsed.hostname + path
    #primerja, če že obstaja obtoječa stran
    if url in websites:
        return False
    websites.add(url)
    print(url)
    return True

is_unique(web)

docs.python.org/3/library


True

In [43]:


url = "https://www.example.com/path/to/page.docx"

import mimetypes
mimetype,encoding = mimetypes.guess_type(url)
mimetype and mimetype.startswith('image')
print(mimetype)
print(encoding)

application/vnd.openxmlformats-officedocument.wordprocessingml.document
None


In [42]:
import urllib 
from bs4 import BeautifulSoup
#pip install bs4
from langdetect import detect
#pip install langdetect
import string
from stopwordsiso import stopwords
import re
from datasketch import MinHash
import numpy as np

In [39]:
def recognizeLang(text):
    """
    Pomožna funkcija, ki pretvori pravilen zapis jezika
    """
    detLang = detect(text)
    

    if detLang == 'zh-cn':
        lang = "zh"
    elif detLang == 'zh-tw':
        lang = "zh"
    else:
        lang = detLang
    return lang



def get_website_text(url_address):
    """
    Funkcija prebere besedilo strani in ga reducira na najmanjše možno obliko
    """

    with urllib.request.urlopen(url_address) as url:
        html = url.read()
        

    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out


    # get text
    text = soup.get_text()
    
    raw_text = soup.get_text().strip().lower()
    # remove unwanted characters such as punctuation marks or special symbols
    raw_text = re.sub(r'[^\w\s]', '', raw_text)
    word_list = []
    for word in raw_text.split():
        word_list.append(word)
        

    # poišče jezik vnešenega teksta iz prvih 100 besed
    reduced_sent = " ".join(word_list[:100])
    lang = recognizeLang(reduced_sent)

  
    
    
    if lang == "th" or lang == "zh" or lang == "vi" or lang == "ja" or lang == "ko":
        #azijci ne delajo presledkov, zato je potrebno obravnavati vsako črko posebej
        asian_words = [c for w in word_list for c in w if not c in stopwords(lang)]
        regex = re.compile('[a-zA-Z]')
        filtered_words = [word for word in asian_words if not regex.search(word)]
    elif not lang == "other":    
        filtered_words = [w for w in word_list if not w in stopwords(lang)]    
    else:
        filtered_words = word_list

    return filtered_words



def generate_minhash(text_list):
    m = MinHash()
    for text in text_list:
        m.update(text.encode('utf8'))
        hashvalues = m.hashvalues
    return hashvalues

def is_page_similar(m1, m2):
    return float(np.count_nonzero(m1==m2)) / float(len(m1))





In [40]:
words1 = get_website_text("https://github.com/kayzhu/LSHash")
hash1 = generate_minhash(words1)
words2 = get_website_text("https://github.com/kayzhu/LSHash/pulls")
hash2 = generate_minhash(words2)
is_page_similar(hash1, hash2)


0.3046875

In [46]:
#pomembno ker nekateri strani brez tega ne dovolijo dostop!
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

url = 'https://okusno.je/'  # Replace example.com with the actual domain
response = requests.get(url, headers=headers)
if response.status_code == 200:  # Check if the response is successful
    #print(len(url_sitemap(url))) #strani je preveč da bi jih editor izpisal zato len
    resp = requests.get(url + '/robots.txt', headers=headers)
    print(robots_context(resp.text))
else:
    print('Failed to retrieve robots.txt')

Disallow: /iskanje
Disallow: /bin/
Disallow: /lbin/
Disallow: /adserver/
Disallow: *modal:*
Crawl-delay: 2
{'User-agent': {'Disallow': ['/iskanje', '/bin/', '/lbin/', '/adserver/', '*modal:*'], 'Allow': []}, 'Crawl-delay': '2', 'Sitemap': ['https://www.24ur.com/sitemaps/sites/10010']}
